## 만개의 레시피 데이터 수집 코드
## https://www.10000recipe.com/recipe/list.html

In [1]:
import pandas as pd
import requests
import time
from tqdm import trange
from bs4 import BeautifulSoup as bs

In [2]:
def get_url(page_no=1):
    url = f'https://www.10000recipe.com/recipe/list.html?order=reco&page={page_no}'
    return url

In [3]:
def get_last_page_no():
    url = get_url()
    response = requests.get(url, verify=False)
    soup = bs(response.text)
    tmp = soup.select('#contents_area_full > ul > div > b')[0].text
    last_page = int(tmp.replace(',', '')) // 40 + 1
    return last_page

In [4]:
def get_one_page(page_no):
    "한 페이지 수집"
    url = get_url(page_no)
    response = requests.get(url, verify=False)
    soup = bs(response.text)
    a_list = soup.select('#contents_area_full > ul > ul > li > div.common_sp_thumb > a')
    link_list = ['http://10000recipe.com' + a['href'] for a in a_list]
    titles = soup.select('#contents_area_full > ul > ul > li > div.common_sp_caption > div.common_sp_caption_tit.line2')
    title_list = [title.text for title in titles]
    tmp = pd.DataFrame({'요리':title_list, '링크':link_list})

    return tmp

In [5]:
def get_all_pages():
    "모든 페이지 수집"
    # last_page = get_last_page_no()
    last_page = 10  # 테스트용 10페이지 지정, 마지막페이지까지 수집하려면 윗줄 실행
    page_list = []
    # 1페이지부터 last까지
    for page in trange(1, last_page+1):
        df_one_page = get_one_page(page)
        page_list.append(df_one_page)
        time.sleep(0.01)
    
    df_all_page = pd.concat(page_list, ignore_index=True)
    return df_all_page

In [6]:
all_page = get_all_pages()
all_page

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
 10%|█         | 1/10 [00:00<00:07,  1.19it/s]/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
 20%|██        | 2/10 [00:01<00:04,  1.72it/s]/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.rea

,요리,링크
0,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,http://10000recipe.com/recipe/6912734
1,"술술 들어가는 쫄순이, 순두부쫄면",http://10000recipe.com/recipe/6989285
2,[살 빠지는 레시피]455kcal 브로콜리 챙겨 먹기 힘들다고요? 닭가슴살 넣고 샐...,http://10000recipe.com/recipe/7001246
3,아삭아삭한 고추된장박이,http://10000recipe.com/recipe/4030952
4,달콤 짭조름♡ 떡 베이컨 간장조림♡,http://10000recipe.com/recipe/6843136
...,...,...
395,추석음식추천]간편한 새우냉채 만들기,http://10000recipe.com/recipe/6986791
396,양송이 치즈구이,http://10000recipe.com/recipe/6835653
397,"채쳐서 바로 무치는 초스피드 밑반찬, 상큼한 양배추 생채",http://10000recipe.com/recipe/6882531
398,초간단 한그릇요리- 고소한 땅콩버터 볶음우동 만들기,http://10000recipe.com/recipe/6990248


In [7]:
def get_ingredients(url):
    try: 
        response = requests.get(url, verify=False)
        soup = bs(response.text)    
        tmp = soup.find('div', 'ready_ingre3')
        ingredients = [i.text.split()[0] for i in tmp.find_all('li')]
        return ingredients
    except:
        return None

In [8]:
url = all_page.iloc[0, -1]
get_ingredients(url)

/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['새송이버섯', '버터', '어린잎채소', '간장', '맛술', '올리고당']

In [9]:
from tqdm.notebook import tqdm
tqdm.pandas()

all_page['재료'] = all_page['링크'].progress_map(get_ingredients)

  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.10000recipe.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/urllib3/connectionp

In [ ]:
# 400개 처리 Colab에서 20분 가량 소요

In [10]:
all_page

,요리,링크,재료
0,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,http://10000recipe.com/recipe/6912734,"[새송이버섯, 버터, 어린잎채소, 간장, 맛술, 올리고당]"
1,"술술 들어가는 쫄순이, 순두부쫄면",http://10000recipe.com/recipe/6989285,"[순두부, 쫄면사리, 달걀, 양파, 대파, 다진마늘, 고춧가루, 물, 간장, 굴소스..."
2,[살 빠지는 레시피]455kcal 브로콜리 챙겨 먹기 힘들다고요? 닭가슴살 넣고 샐...,http://10000recipe.com/recipe/7001246,"[브로콜리, 훈제닭가슴살, 당근, 사과, 마요네즈, 레몬주스, 꿀]"
3,아삭아삭한 고추된장박이,http://10000recipe.com/recipe/4030952,"[풋고추, 된장, 다진, 송송썰은, 참기름, 통깨, 매실원액, 설탕]"
4,달콤 짭조름♡ 떡 베이컨 간장조림♡,http://10000recipe.com/recipe/6843136,"[베이컨, 대파, 떡, 간장, 참기름, 물엿, 기름]"
...,...,...,...
395,추석음식추천]간편한 새우냉채 만들기,http://10000recipe.com/recipe/6986791,"[새우, 오이, 자두, 레몬, 노란파프리카, 빨간파프리카, 다진마늘, 식초, 물, ..."
396,양송이 치즈구이,http://10000recipe.com/recipe/6835653,"[양파, 노랑파프리카, 주황파프리카, 양송이, 토마토스파게티소스, 모짜렐라치즈, 파..."
397,"채쳐서 바로 무치는 초스피드 밑반찬, 상큼한 양배추 생채",http://10000recipe.com/recipe/6882531,"[양배추, 간마늘, 식초, 매실청, 꿀/물엿, 멸치액젓, 국간장(조선간장), 고추가..."
398,초간단 한그릇요리- 고소한 땅콩버터 볶음우동 만들기,http://10000recipe.com/recipe/6990248,"[우동면, 간장, 굴소스, 땅콩버터, 설탕, 볶은, 다진, 고춧가루, 면수, 식물성..."


In [11]:
# 400개 요리 수집 결과 재료가 수집되지 않는 부분 존재
all_page[all_page['재료'].isna()]

,요리,링크,재료
130,떠먹는 떡꼬치 - 때론 떡피자 또는 떡꼬치 새콤 달콤,http://10000recipe.com/recipe/6854230,None


In [15]:
# 재료가 수집되지 않는 부분 삭제
all_page.dropna(subset=['재료'], inplace=True)
all_page

,요리,링크,재료
0,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,http://10000recipe.com/recipe/6912734,"[새송이버섯, 버터, 어린잎채소, 간장, 맛술, 올리고당]"
1,"술술 들어가는 쫄순이, 순두부쫄면",http://10000recipe.com/recipe/6989285,"[순두부, 쫄면사리, 달걀, 양파, 대파, 다진마늘, 고춧가루, 물, 간장, 굴소스..."
2,[살 빠지는 레시피]455kcal 브로콜리 챙겨 먹기 힘들다고요? 닭가슴살 넣고 샐...,http://10000recipe.com/recipe/7001246,"[브로콜리, 훈제닭가슴살, 당근, 사과, 마요네즈, 레몬주스, 꿀]"
3,아삭아삭한 고추된장박이,http://10000recipe.com/recipe/4030952,"[풋고추, 된장, 다진, 송송썰은, 참기름, 통깨, 매실원액, 설탕]"
4,달콤 짭조름♡ 떡 베이컨 간장조림♡,http://10000recipe.com/recipe/6843136,"[베이컨, 대파, 떡, 간장, 참기름, 물엿, 기름]"
...,...,...,...
395,추석음식추천]간편한 새우냉채 만들기,http://10000recipe.com/recipe/6986791,"[새우, 오이, 자두, 레몬, 노란파프리카, 빨간파프리카, 다진마늘, 식초, 물, ..."
396,양송이 치즈구이,http://10000recipe.com/recipe/6835653,"[양파, 노랑파프리카, 주황파프리카, 양송이, 토마토스파게티소스, 모짜렐라치즈, 파..."
397,"채쳐서 바로 무치는 초스피드 밑반찬, 상큼한 양배추 생채",http://10000recipe.com/recipe/6882531,"[양배추, 간마늘, 식초, 매실청, 꿀/물엿, 멸치액젓, 국간장(조선간장), 고추가..."
398,초간단 한그릇요리- 고소한 땅콩버터 볶음우동 만들기,http://10000recipe.com/recipe/6990248,"[우동면, 간장, 굴소스, 땅콩버터, 설탕, 볶은, 다진, 고춧가루, 면수, 식물성..."


In [ ]:
# 필요에 따라 코드 개선 필요